# Fancy Indexing
- using fancy indexing for selecting
- combining indexing method
- using fancy indexing for modifying

In the previous sections, we saw how to access and modify portions of arrays using simple indices (e.g., ``arr[0]``), slices (e.g., ``arr[:5]``), and Boolean masks (e.g., ``arr[arr > 0]``).
In this section, we'll look at another style of array indexing, known as *fancy indexing*.
Fancy indexing is like the simple indexing we've already seen, but we pass arrays of indices in place of single scalars.
This allows us to very quickly access and modify complicated subsets of an array's values.

## Exploring Fancy Indexing

Fancy indexing is conceptually simple: it means passing an array of indices to access multiple array elements at once.
For example, consider the following array:

In [2]:
import numpy as np
rand = np.random.RandomState(42)

x = rand.randint(100, size=10)
x

array([51, 92, 14, 71, 60, 20, 82, 86, 74, 74])

In [4]:
# Suppose we want to access three different elements. We could do it like this:
[x[3], x[7], x[2]]

[71, 86, 14]

In [5]:
# Alternatively, we can pass a single list or array of indices to obtain the same result:
ind = [3, 7, 4]
x[ind]

array([71, 86, 60])

In [12]:
# When using fancy indexing, the shape of the result reflects the shape of the *index arrays* 
#rather than the shape of the *array being indexed*:
ind = np.array([[3, 7],
                [4, 5]])
x[ind]
# note that here X is a 1d array

array([[71, 86],
       [60, 20]])

In [9]:
X = np.arange(12).reshape((3, 4))
X

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

Like with standard indexing, the first index refers to the row, and the second to the column:

In [13]:
row = np.array([0, 1, 2])
col = np.array([2, 1, 3])
X[row, col]
# note that here X is a 2d array

array([ 2,  5, 11])

The pairing of indices in fancy indexing follows all the broadcasting rules.
So, for example, if we combine a column vector and a row vector within the indices, we get a two-dimensional result:

In [16]:
X[row[:, np.newaxis], col] # think about the rule in broadcasting

array([[ 2,  1,  3],
       [ 6,  5,  7],
       [10,  9, 11]])

It is always important to remember with fancy indexing that the return value reflects the *broadcasted shape of the indices*, rather than the shape of the array being indexed.

## Combined Indexing

For even more powerful operations, fancy indexing can be combined with the other indexing schemes we've seen:

In [17]:
print(X)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


We can combine fancy and simple indices:

In [18]:
X[2, [2, 0, 1]]

array([10,  8,  9])

We can also combine fancy indexing with slicing:

In [19]:
X[1:, [2, 0, 1]]

array([[ 6,  4,  5],
       [10,  8,  9]])

And we can combine fancy indexing with masking:

In [22]:
mask = np.array([1, 0, 1, 0], dtype=bool)
X[row[:, np.newaxis], mask]

array([[ 0,  2],
       [ 4,  6],
       [ 8, 10]])

### note the difference between fancy index and mask in the following example.

In [29]:
X2 = X[:, [0, 1]]
X2

array([[0, 1],
       [4, 5],
       [8, 9]])

In [30]:
X2[:, [1,0]]

array([[1, 0],
       [5, 4],
       [9, 8]])

In [31]:
mask = np.array([1, 0], dtype = bool)
X2[:, mask]

array([[0],
       [4],
       [8]])

All of these indexing options combined lead to a very flexible set of operations for accessing and modifying array values.

## Modifying Values with Fancy Indexing

Just as fancy indexing can be used to access parts of an array, it can also be used to modify parts of an array.
For example, imagine we have an array of indices and we'd like to set the corresponding items in an array to some value:

In [38]:
x = np.arange(10)
i = np.array([2, 1, 8, 4])
x[i] = 99
print(x)

[ 0 99 99  3 99  5  6  7 99  9]


We can use any assignment-type operator for this. For example:

In [39]:
x[i] -= 10
print(x)

[ 0 89 89  3 89  5  6  7 89  9]


Notice, though, that repeated indices with these operations can cause some potentially unexpected results. Consider the following:

In [41]:
x = np.zeros(10)
x[[0, 0]] = [8, 888]
print(x)

[888.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


Where did the 4 go? The result of this operation is to first assign ``x[0] = 4``, followed by ``x[0] = 6``.
The result, of course, is that ``x[0]`` contains the value 6.

Fair enough, but consider this operation:

In [42]:
i = [2, 3, 3, 4, 4, 4]
x[i] += 1
x

array([888.,   0.,   1.,   1.,   1.,   0.,   0.,   0.,   0.,   0.])

You might expect that ``x[3]`` would contain the value 2, and ``x[4]`` would contain the value 3, as this is how many times each index is repeated. Why is this not the case?
Conceptually, this is because ``x[i] += 1`` is meant as a shorthand of ``x[i] = x[i] + 1``. ``x[i] + 1`` is evaluated, and then the result is assigned to the indices in x.
With this in mind, it is not the augmentation that happens multiple times, but the assignment, which leads to the rather nonintuitive results.

So what if you want the other behavior where the operation is repeated? For this, you can use the ``at()`` method of ufuncs (available since NumPy 1.8), and do the following:

In [43]:
x = np.zeros(10)
np.add.at(x, i, 1)
print(x)

[0. 0. 1. 2. 3. 0. 0. 0. 0. 0.]


The ``at()`` method does an in-place application of the given operator at the specified indices (here, ``i``) with the specified value (here, 1).
Another method that is similar in spirit is the ``reduceat()`` method of ufuncs, which you can read about in the NumPy documentation.